# Multiverse Parser

Make sure the [Multiverse Framework](https://github.com/Multiverse-Framework/Multiverse) is installed properly.

In [1]:
from pathlib import Path
from typing import Optional, cast
import os
import solara
from solara.website.utils import apidoc

resources_dir = os.path.join(os.getcwd(), "..", "resources")
output_dir = os.path.join(os.getcwd(), "..", "output")

import ipywidgets
from IPython.display import display
import zipfile
import numpy

from multiverse_parser import InertiaSource
from multiverse_parser import UrdfImporter, MjcfImporter, UsdImporter
from multiverse_parser import UrdfExporter, MjcfExporter

out = ipywidgets.Output()
file_index = 0

result_label_widget = ipywidgets.Label("")
usda_widget = ipywidgets.Checkbox(
    value=True, description="USDA", disabled=False, indent=False
)
urdf_widget = ipywidgets.Checkbox(
    value=False, description="URDF", disabled=False, indent=False
)
mjcf_widget = ipywidgets.Checkbox(
    value=False, description="MJCF", disabled=False, indent=False
)
visual_options_widget = ipywidgets.RadioButtons(
    options=["with_visual", "no_visual"],
    value="with_visual",
    description="Visual:",
    disabled=False,
)
collision_options_widget = ipywidgets.RadioButtons(
    options=["with_collision", "no_collision"],
    value="with_collision",
    description="Collision:",
    disabled=False,
)
physics_options_widget = ipywidgets.RadioButtons(
    options=["with_physics", "no_physics"],
    value="with_physics",
    description="Physics:",
    disabled=False,
)
fixed_base_widget = ipywidgets.Checkbox(
    value=True, description="Fix based", disabled=False, indent=False
)
inertiasource_options_widget = ipywidgets.ToggleButtons(
    options=["From source", "From visual meshes", "From collision meshes"],
    description="Inertia source:",
    disabled=False,
    button_style="",
)
add_xform_for_each_geom_widget = ipywidgets.Checkbox(
    value=False,
    description="Add body for each geom (only for input USD)",
    disabled=False,
    indent=False,
)
default_color_widget = ipywidgets.ColorPicker(
    concise=False, description="Color", value="red", disabled=False
)
default_collision_opacity_widget = ipywidgets.FloatSlider(
    value=0.0,
    min=0,
    max=1.0,
    step=0.1,
    description="Opacity:",
    disabled=False,
    continuous_update=False,
    orientation="horizontal",
    readout=True,
    readout_format=".1f",
)
hbox_options_widget = ipywidgets.VBox(
    [
        ipywidgets.HBox(
            [
                ipywidgets.Label("Export as:"),
                ipywidgets.VBox([usda_widget, urdf_widget, mjcf_widget]),
            ]
        ),
        ipywidgets.HBox(
            [visual_options_widget, collision_options_widget, physics_options_widget]
        ),
        ipywidgets.VBox(
            [
                fixed_base_widget,
                add_xform_for_each_geom_widget,
                inertiasource_options_widget,
            ]
        ),
        ipywidgets.Label("Default mesh RGBA:"),
        ipywidgets.HBox([default_color_widget, default_collision_opacity_widget]),
    ]
)

uploader_widget = ipywidgets.FileUpload(
    accept=".zip", multiple=False, description="Upload ZIP file"
)


def hex_to_rgb(hex_color):
    # Remove the '#' character if present
    hex_color = hex_color.lstrip("#")

    # Convert hex to RGB tuple
    rgb_color = tuple(int(hex_color[i : i + 2], 16) for i in (0, 2, 4))

    # Normalize the RGB values to the range 0-1
    rgb_normalized = tuple(c / 255.0 for c in rgb_color)

    return rgb_normalized


def zip_directory(directory_path, output_zip):
    with zipfile.ZipFile(output_zip, "w") as zipf:
        for foldername, subfolders, filenames in os.walk(directory_path):
            for filename in filenames:
                # Create the complete filepath of the file in the directory
                file_path = os.path.join(foldername, filename)
                # Add file to zip
                zipf.write(file_path, os.path.relpath(file_path, directory_path))


@out.capture()
def convert(input_file_path):
    if input_file_path is None:
        return
    input_file_path = str(input_file_path)
    print(f"Convert {input_file_path}")
    in_extension = os.path.splitext(input_file_path)[1]
    fixed_base = fixed_base_widget.value
    with_physics = physics_options_widget.value == "with_physics"
    with_visual = visual_options_widget.value == "with_visual"
    with_collision = collision_options_widget.value == "with_collision"
    rgb = default_color_widget.value
    if inertiasource_options_widget.value == "From visual meshes":
        inertia_source = InertiaSource.FROM_VISUAL_MESH
    elif inertiasource_options_widget.value == "From collision meshes":
        inertia_source = InertiaSource.FROM_COLLISION_MESH
    elif inertiasource_options_widget.value == "From source":
        inertia_source = InertiaSource.FROM_SRC

    if rgb == "red":
        r = 1.0
        g = 0.0
        b = 0.0
    else:
        r, g, b = hex_to_rgb(rgb)

    a = float(default_collision_opacity_widget.value)
    default_rgba = numpy.array([r, g, b, a])

    if in_extension in [".usd", ".usda", ".usdc"]:
        factory = UsdImporter(
            file_path=input_file_path,
            fixed_base=fixed_base,
            with_physics=with_physics,
            with_visual=with_visual,
            with_collision=with_collision,
            inertia_source=inertia_source,
            default_rgba=default_rgba,
            add_xform_for_each_geom=add_xform_for_each_geom_widget.value,
        )
    elif in_extension == ".urdf":
        factory = UrdfImporter(
            file_path=input_file_path,
            fixed_base=fixed_base,
            with_physics=with_physics,
            with_visual=with_visual,
            with_collision=with_collision,
            inertia_source=inertia_source,
            default_rgba=default_rgba,
        )
    elif in_extension == ".xml":
        factory = MjcfImporter(
            file_path=input_file_path,
            fixed_base=fixed_base,
            with_physics=with_physics,
            with_visual=with_visual,
            with_collision=with_collision,
            inertia_source=inertia_source,
            default_rgba=default_rgba,
        )
    else:
        raise NotImplementedError(
            f"Importing from {in_extension} is not supported yet."
        )

    factory.import_model()

    output_file_name = os.path.splitext(os.path.basename(input_file_path))[0]

    output_dir_path = os.path.join(output_dir, output_file_name)
    output_exts = ""
    if usda_widget.value:
        output_usd_path = os.path.join(output_dir_path, output_file_name + ".usda")
        factory.save_tmp_model(usd_file_path=str(output_usd_path))
        output_exts += "USDA, "
    if mjcf_widget.value:
        output_xml_path = os.path.join(output_dir_path, output_file_name + ".xml")
        exporter = MjcfExporter(file_path=output_xml_path, factory=factory)
        exporter.build()
        exporter.export(keep_usd=False)
        output_exts += "MJCF, "
    if urdf_widget.value:
        output_urdf_path = os.path.join(output_dir_path, output_file_name + ".urdf")
        exporter = UrdfExporter(file_path=output_urdf_path, factory=factory)
        exporter.build()
        exporter.export(keep_usd=False)
        output_exts += "URDF, "

    if output_exts == "":
        return

    output_exts = output_exts[:-2]
    global file_index
    output_zip_name = f"{output_file_name}_{file_index}.zip"
    file_index += 1

    output_zip_path = os.path.join("/tmp", output_zip_name)
    zip_directory(output_dir_path, output_zip_path)

    display(
        ipywidgets.VBox(
            [
                solara.FileDownload.widget(
                    data=open(output_zip_path, "rb").read(),
                    label=f"Download ZIP (includes {output_exts})",
                    filename=output_zip_name,
                )
            ]
        )
    )


@solara.component
def FileBrowser():
    input_file_path, set_file = solara.use_state(cast(Optional[Path], None))
    path, set_path = solara.use_state(cast(Optional[Path], None))
    directory, set_directory = solara.use_state(Path(resources_dir).expanduser())

    with solara.VBox() as main:
        can_select = True

        def reset_path():
            set_path(None)
            set_file(None)

        # reset path and file when can_select changes
        solara.use_memo(reset_path, [can_select])
        solara.Info(
            f"Select a scene description file to convert (URDF, XML(MJCF) or USDA"
        )
        solara.FileBrowser(
            directory,
            on_directory_change=set_directory,
            on_path_select=set_path,
            on_file_open=convert,
            can_select=can_select,
        )
        solara.Info(f"Convert file: {input_file_path}")

    return main


with out:
    display(hbox_options_widget, FileBrowser())

out


Output()